In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')
df_=pd.read_csv('test.csv')

In [3]:
df2 = pd.concat([df, df_])

In [4]:
df2.dtypes

Description                          object
Commenting                            int64
Ogling/Facial Expressions/Staring     int64
Touching /Groping                     int64
dtype: object

In [5]:
import numpy as np

In [6]:
df2.isnull().sum()

Description                          0
Commenting                           0
Ogling/Facial Expressions/Staring    0
Touching /Groping                    0
dtype: int64

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8902 entries, 0 to 1700
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Description                        8902 non-null   object
 1   Commenting                         8902 non-null   int64 
 2   Ogling/Facial Expressions/Staring  8902 non-null   int64 
 3   Touching /Groping                  8902 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 347.7+ KB


In [8]:
!pip install nltk

In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Pre-processing

1) Converting text to Lower case
2) Removal of digits
3) Removal of punctuations
4) Removal of special characters
5) Removal of html tags
6) Removal of stop words
---



In [10]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    return ' '.join(tokens)

In [12]:
test_data_after_processing=[]

In [13]:
import re

In [14]:
for item in df2.Description:
  test_data_after_processing.append(preprocess_text(item))

In [15]:
df2['Description_after_processing']=test_data_after_processing

In [16]:
df2.head()

,Description,Commenting,Ogling/Facial Expressions/Staring,Touching /Groping,Description_after_processing
0,"Was walking along crowded street, holding mums...",0,0,1,walking along crowded street holding mums hand...
1,This incident took place in the evening.I was ...,0,1,0,incident took place metro two guys started sta...
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1,0,0,waiting bus man came bike offering liftvto you...
3,Incident happened inside the train,0,0,0,incident happened inside train
4,I witnessed an incident when a chain was bruta...,0,0,0,witnessed incident chain brutally snatched eld...


In [17]:
pip install textblob

In [18]:
from textblob import TextBlob

In [19]:
pol=[]


In [20]:
for item in df2.Description_after_processing:
  blob=TextBlob(item)
  pol.append(blob.sentiment.polarity)

In [21]:
for i in range(len(pol)):
  if pol[i]<=-0.4:
    pol[i]=1   #Sexual
  else:
    pol[i]=0   #Non-Sexual

In [22]:
df2['pol']=pol

In [23]:
df2.head()

,Description,Commenting,Ogling/Facial Expressions/Staring,Touching /Groping,Description_after_processing,pol
0,"Was walking along crowded street, holding mums...",0,0,1,walking along crowded street holding mums hand...,0
1,This incident took place in the evening.I was ...,0,1,0,incident took place metro two guys started sta...,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1,0,0,waiting bus man came bike offering liftvto you...,0
3,Incident happened inside the train,0,0,0,incident happened inside train,0
4,I witnessed an incident when a chain was bruta...,0,0,0,witnessed incident chain brutally snatched eld...,1


In [24]:
df2=df2.drop(['Description','Commenting','Ogling/Facial Expressions/Staring','Touching /Groping'],axis=1)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
def text_splitter(text):

    return text.split()



tfidf = TfidfVectorizer(
                        min_df=0.00009,
                        max_features=2000,
                        smooth_idf=True,
                        norm="l2",
                        tokenizer = text_splitter,
                        sublinear_tf=False,
                        ngram_range=(1,3)
                       )

tfidf = tfidf.fit_transform(df2.Description_after_processing.apply(lambda x: np.str_(x)))




/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [26]:
x=tfidf
y=df2.pol

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred)

0.961257720381808

In [32]:
# !pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 4.2 MB/s eta 0:00:00


In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred)

0.9545199326221224

In [30]:
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred)

0.9472206625491297

In [31]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
model=LGBMClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred)

[LightGBM] [Info] Number of positive: 574, number of negative: 6547
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.085635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21489
[LightGBM] [Info] Number of data points in the train set: 7121, number of used features: 816
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080607 -> initscore=-2.434133
[LightGBM] [Info] Start training from score -2.434133


0.9629421673217293